In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib
from  matplotlib import pyplot as plt
%matplotlib qt
import scipy
import librosa
import librosa.display
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import FVA.FVA as fva
from FVA.lpc import *
from FVA.detect import *
from FVA.mfcc import *
from FVA.animation import *
from FVAmanager import *
from DB.DBmanager import *

In [6]:
bias  = "FVA/target_sounds/"
name  = "recorded.wav"
FM  =   FVAmanager(bias+name)
fm  =   FM.FR_formants()
fp  =   FM.FR_pitch()
print(fm,fp,sep='\n')
dellist = []
for i,x in enumerate(fm):
    if x < fp + 50 and x > fp - 50: dellist.append(i)
for i in dellist:
    del fm[i]

13/13 [==============================] - 3s 244ms/step
CREPE: Saved the estimated frequencies and confidence values at FVA/target_sounds/recorded.f0.csv
[70.81274703557312, 1402.637104743083, 3389.479372529644, 5184.310153162055, 5769.877099802371, 6215.180335968379, 6645.50395256917, 7019.994441699604, 7365.887475296442, 7590.581768774703, 9625.086462450592, 10570.164278656126]
59.233


In [7]:
print(fm,fp,sep='\n')

[1402.637104743083, 3389.479372529644, 5184.310153162055, 5769.877099802371, 6215.180335968379, 6645.50395256917, 7019.994441699604, 7365.887475296442, 7590.581768774703, 9625.086462450592, 10570.164278656126]
59.233


In [2]:
formants        =   pd.read_excel("DB/sounds.xlsx")
nF1, nF2        =   10, 10
result, users   =   getDB()
formants        =   pd.read_excel("DB/sounds.xlsx")
vowels          =   {'ㅏ':0,'ㅓ':1,'ㅗ':2,'ㅜ':3,'ㅡ':4,'ㅣ':5,'ㅐ':6,'ㅔ':7}
colors          =   ['red','green','orange','dodgerblue','deeppink','aqua','blueviolet','yellow']
counts          =   np.zeros((8,nF1,nF2))
userDict        =   {a[0]:a[1] for a in users.values.tolist()}

In [3]:
def toymd(x):
   return x.split('T')[0]

def countSetup(namefilter=None,timefilter=None):
   """
   Args:
      namefilter (list): [names]
      timefilter (list): [start, end] with "2023-10-03" style
   """
   counts = np.zeros((8,nF1,nF2))
   for x in result.values.tolist():
      if (namefilter is None) or (userDict.get(x[4],None) in namefilter):
         if (timefilter is None) or (timefilter[0]<=toymd(x[3]) and timefilter[0]<=toymd(x[3])):
            counts[vowels[x[2]]][int(x[1].split('_')[0])-1][int(x[1].split('_')[1].split('.')[0])-1] += 1
   
   return counts.transpose(1,2,0)

In [4]:
NameFilter = ['노수빈']
TimeFilter = None#['2023-09-27','2023-10-03']

board = countSetup(namefilter=NameFilter,timefilter=TimeFilter)
print(board.sum())

1225.0


In [8]:
F1 = [float(formants[i+1][0]) for i in range(10)]
F2 = [float(formants[i+1][1]) for i in range(10)]
logF1 = np.log10(np.array(F1))
logF2 = np.log10(np.array(F2))
nwF1 = np.log10(np.array([fm[0]]))[0]
nwF2 = np.log10(np.array([fm[1]]))[0]
nwF1in  = -1
nwF1v   = 987654321.
nwF2in  = -1
nwF2v   = 987654321.
for i in range(10):
  if nwF1v> abs(logF1[i]-nwF1):
    nwF1v = abs(logF1[i]-nwF1)
    nwF1in= i
  if nwF2v> abs(logF2[i]-nwF2):
    nwF2v = abs(logF2[i]-nwF2)
    nwF2in= i
print(F1[nwF1in],F2[nwF2in])

1200.0 3500.0


In [9]:
print({v:k for k,v in vowels.items()}[board[nwF1in][nwF2in].argmax()])

ㅐ


In [5]:
def softmax(x): # numpy array!!
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def probability(a):
    sum = a.sum()
    return a/sum

In [10]:
board[nwF1in][nwF2in]

array([0., 0., 0., 0., 0., 0., 9., 0.])

In [11]:
a = np.array(board[nwF1in][nwF2in])
b = probability(a)
a = softmax(a)

precision = 1
pairs_a = [[round(a[i]*100,precision),list(vowels.keys())[i]] for i in range(len(vowels.items()))]
pairs_b = [[round(b[i]*100,precision),list(vowels.keys())[i]] for i in range(len(vowels.items()))]
pairs_b = sorted(pairs_b,key=lambda x:x[0],reverse=True)

nVW = 4
for x in pairs_b[:nVW]:
    print(x[1],": ",x[0],"%",sep='',end='\n')
    
print(f"Pitch : {round(fp,1)}Hz")

print(f'Formants : ',end='')
nFM = 4
for i,x in enumerate(fm[:nFM]):
    print(round(x,1),"Hz",sep='',end='\n' if i+1==nFM else ', ')

ㅐ: 100.0%
ㅏ: 0.0%
ㅓ: 0.0%
ㅗ: 0.0%
Pitch : 59.2Hz
Formants : 1402.6Hz, 3389.5Hz, 5184.3Hz, 5769.9Hz
